In [1]:
REQUEST_ID = 198
AOI = 'POLYGON ((-94.74437712304278 42.10688312211514, -94.78351591698579 42.10713783113902, -94.78351591698579 42.13795006926095, -94.74437712304278 42.13756819109344, -94.74437712304278 42.10688312211514))'
START_DATE = '2020-05-01'
END_DATE = '2020-09-31'

In [2]:
!pip3 install -q tslearn

In [3]:
import os
import shutil
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio import features
from rasterio.merge import merge
from rasterio.warp import (aligned_target,
                           calculate_default_transform,
                           reproject,
                           Resampling)
from shapely.geometry import Polygon, shape, LinearRing, box
import shapely.wkt
from pathlib import Path
from datetime import datetime
import matplotlib.pylab as plt

from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds, transform_resolution
from code.download.load_tiles import load_images
from code.index_research import combine_bands, calculate_ndvi, calculate_tcari, calculate_msr_g

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

/opt/conda/lib/python3.7/site-packages/tslearn/clustering/kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


In [4]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [5]:
SEED = 66
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/pbdnn")
PBD_DIR = os.path.join(BASE, "notebooks/pbdnn")

BANDS = {'B04','B08', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

for pbd_file in os.listdir(os.path.join(BASE, 'results/pbdnn')):
    if pbd_file.startswith(str(REQUEST_ID)+'_') or pbd_file.startswith('demo_'+str(REQUEST_ID)+'_'):
        PB_PATH = os.path.join(BASE, 'results/pbdnn', pbd_file)
        break
        
    else:
        # should be trigger PBDNN notebook excecution? PBDNN pipeline?
        PB_PATH = aoi_filename
        
PB_PATH

'/home/jovyan/work/results/pbdnn/188_2020-06-11_2020-07-13.geojson'

In [6]:
PB_PATH = aoi_filename

In [7]:
year = START_DATE.split('-')[0]

starts = [
    f'{year}-05-01', f'{year}-05-08', f'{year}-05-15', f'{year}-05-22',
    f'{year}-06-01', f'{year}-06-08', f'{year}-06-15', f'{year}-06-22', 
    f'{year}-07-01', f'{year}-07-08', f'{year}-07-15', f'{year}-07-22',
    f'{year}-08-01', f'{year}-08-08', f'{year}-08-15', f'{year}-08-22',
]

ends = [
    f'{year}-05-07', f'{year}-05-14', f'{year}-05-21', f'{year}-05-30',
    f'{year}-06-07', f'{year}-06-14', f'{year}-06-21', f'{year}-06-30', 
    f'{year}-07-07', f'{year}-07-14', f'{year}-07-21', f'{year}-07-30',
    f'{year}-08-07', f'{year}-08-14', f'{year}-08-21', f'{year}-08-30'
]

b04_tiles, b08_tiles, tci_tiles = [], [], []

date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)

In [8]:
for start, end in zip(starts, ends):
    loadings = load_images(API_KEY, date_tile_info.tileID.values, start, end, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
    checked = check_nodata(loadings, PRODUCT_TYPE)

    try:
        checked = get_min_clouds(checked)
    except Exception as e:
        print(f'No clean raster found for period from {start} to {end}, skipping, {e}')

    for i, tile in date_tile_info.iterrows():
        try:
            tile_folder = Path(checked[tile.tileID])
            print(f'filtered: {tile_folder}')
        except Exception as ex:
            print(ex)
            continue

        b04_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B04_10m.jp2' in filename][0]
        b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
        tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

        b04_tiles.append(b04_tile)
        b08_tiles.append(b08_tile)
        tci_tiles.append(tci_tile)

No clean raster found for period from 2020-05-01 to 2020-05-07, skipping, list index out of range
expected str, bytes or os.PathLike object, not set
No clean raster found for period from 2020-05-08 to 2020-05-14, skipping, list index out of range
expected str, bytes or os.PathLike object, not set
No clean raster found for period from 2020-05-15 to 2020-05-21, skipping, list index out of range
expected str, bytes or os.PathLike object, not set
filtered: /home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200529T170849_N0214_R112_T15TUG_20200529T212318
filtered: /home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200603T170901_N0214_R112_T15TUG_20200603T231006
filtered: /home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200608T170849_N0214_R112_T15TUG_20200608T211003
No clean raster found for period from 2020-06-15 to 2020-06-21, skipping, list index out of range
expected str, bytes or os.PathLike object, not set
No clean raster found for period from 2020-06-22 to 2020-06-30, skipping, list inde

In [9]:
b04_tiles

['/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200529T170849_N0214_R112_T15TUG_20200529T212318/T15TUG_20200529T170849_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200603T170901_N0214_R112_T15TUG_20200603T231006/T15TUG_20200603T170901_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200608T170849_N0214_R112_T15TUG_20200608T211003/T15TUG_20200608T170849_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200713T170901_N0214_R112_T15TUG_20200713T213310/T15TUG_20200713T170901_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200718T170849_N0214_R112_T15TUG_20200718T212228/T15TUG_20200718T170849_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200728T170849_N0214_R112_T15TUG_20200728T212118/T15TUG_20200728T170849_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200817T170849_N0214_R112_T15TUG_20200817T200143/T15TUG_20200817T170849_B04_10m.jp2',
 '/home/jovyan/work/satellite_imagery/S2B_MSIL2A

In [10]:
with rasterio.open(tci_tiles[0]) as src:
    tci_image = src.read()

In [11]:
for i in tqdm(range(len(b04_tiles))):
    os.makedirs(f'{BASE}/results/dd/plant_stress/{REQUEST_ID}', exist_ok=True)
    ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/ndvi_{i}.tif'
    if not os.path.exists(ndvi_path):
        calculate_ndvi(b04_tiles[i], b08_tiles[i], out_path=ndvi_path, nodata=np.nan)

In [12]:
from ipywidgets import interact
import ipywidgets as widgets

r_len = len(b04_tiles)

def plot_images(tci_imgs, ndvi_imgs, pred_imgs, anom_images):
    fig, ax = plt.subplots(len(tci_imgs), 4, figsize=(17, len(tci_imgs)*2))

    for i, axes in zip(range(len(tci_imgs)), ax):
        
        if tci_imgs[i].shape[0] == 3:
            axes[0].imshow(tci_imgs[i].transpose(1, 2, 0))
        else:
            axes[0].imshow(tci_imgs[i], cmap='gray') #np.where(gt_imgs[i]==tp, gt_imgs[i], 0))
        axes[1].imshow(ndvi_imgs[i], cmap='gray') #np.where(gt_imgs[i]==fp, gt_imgs[i], 0))
        axes[2].imshow(pred_imgs[i], cmap='gray') #np.where(gt_imgs[i]==fn, gt_imgs[i], 0))
        axes[3].imshow(anom_images[i], cmap='gray')
    
    ax[0][0].set_title('TCI');
    ax[0][1].set_title('NDVI');
    ax[0][2].set_title('Clusters');
    ax[0][3].set_title('Prediction');
    
        
    plt.tight_layout();
    return tci_imgs, ndvi_imgs, pred_imgs, anom_images

def visualize_results(tci_images, ndvi_imgs, pred_ndvi, results):
    
    def get_arrays(num):
        tci = tci_images[r_len*num:r_len*num+r_len]
        ndvi = ndvi_imgs[r_len*num:r_len*num+r_len]
        pred = pred_ndvi[r_len*num:r_len*num+r_len]
        result = results[r_len*num:r_len*num+r_len]
        
        return tci, ndvi, pred, result

    def f_fabric(num):
        
        def f(x):
        
            tci, ndvi, pred, result = get_arrays(num)
            plot_images(tci, ndvi, pred, result)
            
        return f
    
    plt.rcParams.update({'font.size': 14})
    f = f_fabric(0)
    interact(f, x=widgets.IntSlider(min=0, max=(len(tci_images)//r_len), step=1, value=0, continuous_update=False))

In [13]:
def transform_crs(data_path, save_path, dst_crs='EPSG:4326', resolution=(10, 10)):
    with rasterio.open(data_path) as src:
        if resolution is None:
            transform, width, height = calculate_default_transform(src.crs,
                                                                   dst_crs,
                                                                   src.width,
                                                                   src.height,
                                                                   *src.bounds)
        else:
            transform, width, height = calculate_default_transform(src.crs,
                                                                   dst_crs,
                                                                   src.width,
                                                                   src.height,
                                                                   *src.bounds,
                                                                   resolution=resolution)
        kwargs = src.meta.copy()
        kwargs.update({'crs': dst_crs,
                       'transform': transform,
                       'width': width,
                       'height': height})
        with rasterio.open(save_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
        dst.close()
    src.close()

    return save_path

In [14]:
def stitch_tiles(paths, out_raster_path='test.tif'):
    tiles = []
    tmp_files = []
    
    for i, path in enumerate(paths):
        if i == 0:
            file = rasterio.open(path)
            meta, crs = file.meta, file.crs
        else:
            tmp_path = path.replace(
                '.jp2', '_tmp.jp2').replace('.tif', '_tmp.tif')
            crs_transformed = transform_crs(path, tmp_path, 
                                            dst_crs=crs, 
                                            resolution=None)
            tmp_files.append(crs_transformed)
            file = rasterio.open(crs_transformed)
        tiles.append(file)
            
    tile_arr, transform = merge(tiles, method='last')
    
    
    meta.update({"driver": "GTiff",
                 "height": tile_arr.shape[1],
                 "width": tile_arr.shape[2],
                 "transform": transform,
                 "crs": crs})
    
    if '.jp2' in out_raster_path:
        out_raster_path = out_raster_path.replace('.jp2', '_merged.tif')
    else:
        out_raster_path = out_raster_path.replace('.tif', '_merged.tif')
    print(f'saved raster {out_raster_path}')

    for tile in tiles:
        tile.close()
        
    for tmp_file in tmp_files:
        try:
            os.remove(tmp_file)
        except FileNotFoundError:
            print(f'Tile {tmp_file} was removed or renamed, skipping')
        
    with rasterio.open(out_raster_path, "w", **meta) as dst:
        dst.write(tile_arr)
    
    return out_raster_path

In [15]:
fields = gpd.read_file(PB_PATH)

In [62]:
all_polys = gpd.GeoDataFrame()
num_clusters = 2
conf = 0.5
z_score = 2

tci_images, ndvi_imgs, pred_ndvi, results = [], [], [], []
rasters_to_stitch, steps_to_vis = [], []
p_bar = tqdm(fields.iterrows(), total=len(fields))

for idx, row in p_bar:
    
    ndvi_steps = []
    tci_steps = []
    clustered = []
    
    kmeans = TimeSeriesKMeans(n_clusters=2, metric="softdtw",
                              max_iter=10, random_state=SEED)
    
    for i in range(len(b04_tiles)):
        
        tci_path = tci_tiles[i]
        ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/ndvi_{i}.tif'
    
        with rasterio.open(tci_path) as src:
            tci_image, tfs = riomask.mask(
                src, [fields.to_crs(src.crs).iloc[idx].geometry], all_touched=False, crop=True)

        with rasterio.open(ndvi_path) as src:
            ndvi, tfs = riomask.mask(
                src, [fields.to_crs(src.crs).iloc[idx].geometry], all_touched=False, crop=True)
            crs = src.crs
            meta = src.meta
            
        mu, sigma = ndvi.mean(), ndvi.std()
        #print(mu, sigma)
        
        lower_bound, upper_bound = mu - z_score*sigma, mu + z_score*sigma
        
        
        raster_mask = ndvi.astype(np.bool)
        ndvi = np.where( (ndvi > lower_bound) & (ndvi < upper_bound), ndvi, 0 )
        ndvi_img = np.where(raster_mask, ndvi, 0)
        ndvi_step = ndvi_img[raster_mask]
            
        ndvi_steps.append(ndvi_step)
        ndvi_imgs.append(ndvi[0])
        tci_images.append(tci_image)
    # break
        
    
    try:
        timesteps = np.stack([x.ravel() for x in ndvi_steps], axis=-1) #.reshape(len(b04_tiles), -1)
    except Exception as e:
        print(e, '\n', [x.shape for x in ndvi_steps])
        break
        continue
    kmeans.fit(timesteps)
    
    c, h, w = ndvi.shape
    labels_ = kmeans.labels_     

    ndvi_pred = ndvi_img.ravel()
    ndvi_pred[raster_mask.ravel()] = labels_ + 1
    ndvi_pred = ndvi_pred.reshape(h, w)

    label_1 = np.where(ndvi_pred==1, ndvi[0], 0)
    label_2 = np.where(ndvi_pred==2, ndvi[0], 0)
    anomaly_label = 1 if label_1.mean() < label_2.mean() else 2
    
    for i in range(len(b04_tiles)):
        pred_ndvi.append(ndvi_pred)

    for i in range(len(b04_tiles)):
        results.append(np.where(ndvi_pred==anomaly_label, 1, 0).astype(np.uint8))
    
    raster_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/{REQUEST_ID}_field_{idx}.tif'
    # raster_path = f'{BASE}/results/dd/{REQUEST_ID}_field_{idx}.tif'
    
    colors = [(0, 0, 0), (182, 10, 28),]

    labels = []
    result_3c = ndvi_img.reshape(1, ndvi.shape[-2], ndvi.shape[-1])
    mask = np.zeros((result_3c[0].shape[-2], result_3c[0].shape[-1], 3))
    
    for ix, color in enumerate(colors):

        mask[result_3c[0]==ix] = color

        labels.append({
            "color": ",".join(str(color).split(',')),
        })
        
    meta['height'] = ndvi.shape[-2]
    meta['width'] = ndvi.shape[-1]
    meta['transform'] = tfs
    meta['dtype'] = rasterio.uint8
    
    meta.update(
        count=3,
        nodata=0,
        compress='lzw',
        photometric='RGB'
    )
    
    p_bar.set_postfix({"raster": raster_path, "mu": mu, "sigma":sigma, "anomaly": anomaly_label})
    if os.path.exists(raster_path):
        continue
    with rasterio.open(raster_path, 'w', **meta) as dst:

        

        dst.update_tags(start_date=START_DATE, 
                        end_date=END_DATE, 
                        request_id=str(REQUEST_ID)+'_'+str(ix),
                        labels=labels,
                        name=f'Field {idx} anomalies')

        for i in range(mask.shape[-1]):
            dst.write(mask[:,:,i].astype(np.uint8), indexes=i+1)
            
    rasters_to_stitch.append(raster_path)

In [63]:
colors = ['red', 'blue']

def visualize_values(tci_imgs, ndvi_imgs, pred_imgs, colors=colors):
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 8))
    num_clusters = len(np.unique(pred_imgs)) - 1
    
    for cl in range(num_clusters):
        mask = np.where(pred_imgs[0]==cl+1, True, False)
        ndvi_series = [ndvi_imgs[i][mask] for i in range(len(ndvi_imgs))]
        ax[0].plot(ndvi_series, color=colors[cl], alpha=0.3);
        ax[1].plot(np.mean(ndvi_series, axis=-1), color=colors[cl], label=cl+1)
        ax[1].legend();
        
    plt.tight_layout();

In [64]:
len(tci_images), len(pred_ndvi), len(results)

(72, 72, 72)

In [65]:
pred_ndvi[0].shape

(43, 40)

In [84]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        shutil.rmtree(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')
        
    for i in range(len(b04_tiles)):
        os.makedirs(f'{BASE}/results/dd/plant_stress/{REQUEST_ID}', exist_ok=True)
        ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/ndvi_{i}.tif'
        try:
            os.remove(ndvi_path)
        except FileNotFoundError:
            print(f'Error removing file {ndvi_path}')
            
    #for raster_path in rasters_to_stitch:
    #    try:
    #        os.remove(raster_path)
    #    except FileNotFoundError:
    #        print(f'Error removing file {raster_path}')